In [106]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
import warnings


In [107]:
df = pd.read_csv('data/stud.csv')

In [108]:
df.head()


,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [109]:
df['average'] = df[['math_score', 'reading_score', 'writing_score']].mean(axis=1)
df['average'] = df['average'].round(2)
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score,average
0,female,group B,bachelor's degree,standard,none,72,72,74,72.67
1,female,group C,some college,standard,completed,69,90,88,82.33
2,female,group B,master's degree,standard,none,90,95,93,92.67
3,male,group A,associate's degree,free/reduced,none,47,57,44,49.33
4,male,group C,some college,standard,none,76,78,75,76.33


In [110]:
X = df.drop(columns = ['average'], axis = 1)

In [111]:
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [112]:
y = df.average
y.head()

0    72.67
1    82.33
2    92.67
3    49.33
4    76.33
Name: average, dtype: float64

In [113]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

# Select numeric and categorical features
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

# Drop the "math" column from the categorical features
num_features = num_features.drop('math_score')  # Drop the "math" column

# Define the transformers
num_transfromer = StandardScaler()
cat_transfromer = OneHotEncoder()

# Create the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('OneHotEncoder', cat_transfromer, cat_features),
        ('StandardScaler', num_transfromer, num_features)
    ]
)


In [114]:
# Select numeric and categorical features
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

In [115]:
X = preprocessor.fit_transform(X)

In [116]:
X[0]

array([1.        , 0.        , 0.        , 1.        , 0.        ,
       0.        , 0.        , 0.        , 1.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 1.        ,
       0.        , 1.        , 0.19399858, 0.39149181])

In [117]:
X.shape

(1000, 19)

In [118]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 42)


In [119]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true ,predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [120]:
model ={
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(), 
    "AdaBoost Regressor": AdaBoostRegressor()
}

model_list =[]
r2_list = []

for model_name,model in model.items():
    model.fit(X_train,y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Evaluate Train and Test dataset
    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    # Print model performance
    print(model_name)
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))
    print('----------------------------------')
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))

    model_list.append(model_name)
    r2_list.append(model_test_r2)

    print('=' * 35)
    print('\n')




Linear Regression
Model performance for Training set
- Root Mean Squared Error: 1.7749
- Mean Absolute Error: 1.4224
- R2 Score: 0.9842
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 1.7982
- Mean Absolute Error: 1.4053
- R2 Score: 0.9849


Lasso
Model performance for Training set
- Root Mean Squared Error: 3.0491
- Mean Absolute Error: 2.4353
- R2 Score: 0.9534
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 3.1373
- Mean Absolute Error: 2.5633
- R2 Score: 0.9541


Ridge
Model performance for Training set
- Root Mean Squared Error: 1.7747
- Mean Absolute Error: 1.4218
- R2 Score: 0.9842
----------------------------------
Model performance for Test set
- Root Mean Squared Error: 1.7968
- Mean Absolute Error: 1.4039
- R2 Score: 0.9849


K-Neighbors Regressor
Model performance for Training set
- Root Mean Squared Error: 3.2689
- Mean Absolute Error: 2.5662
- R2 Score: 0.9464
-----------------------

In [121]:
result_df = pd.DataFrame({
    'Models': model_list,
    'r2_score': r2_list
})

In [122]:
result_df.head()

,Models,r2_score
0,Linear Regression,0.984917
1,Lasso,0.954085
2,Ridge,0.984940
3,K-Neighbors Regressor,0.912842
4,Decision Tree,0.949672


In [123]:
result_df = result_df.sort_values(by="r2_score", ascending=False)
result_df.head()

,Models,r2_score
2,Ridge,0.984940
0,Linear Regression,0.984917
6,XGBRegressor,0.975830
5,Random Forest Regressor,0.973294
7,AdaBoost Regressor,0.960599


In [124]:
lin_model = LinearRegression(fit_intercept=True)
lin_model = lin_model.fit(X_train, y_train)
y_pred = lin_model.predict(X_test)
score = r2_score(y_test, y_pred)*100
print(" Accuracy of the model is %.2f" %score)

 Accuracy of the model is 98.49
